In [1]:
import rioxarray as rioxr
import os
import numpy as np

from rasterio.crs import CRS
import geopandas as gpd

from shapely.geometry import box

import rasterio

In [2]:
year = 2020

In [3]:
# directory where the NAIP classifications are
root = '/home/jovyan/msai4earth-esa/iceplant_detection/temp/' + os.path.join('LS_filter_clip_preds_' + str(year))

In [17]:
def make_command(out_file, in_files):
    return 'gdal_merge.py -ot Int16 -of GTiff -a_nodata 100 ' + '-o ' + out_file + ' ' + in_files

In [3]:
# the two CRS that the NAIP scenes have
crs11 = CRS.from_epsg(26911)
crs10 = CRS.from_epsg(26910)

In [14]:
# list of files in that directory
tifs = os.listdir(root)

# directories for scenes with crs= epsg(26911) and rs= epsg(26910)
fp_11 = os.path.join(root, 'crs_26911')
fp_10 = os.path.join(root, 'crs_26910')

# make directories
os.mkdir(fp_10)
os.mkdir(fp_11)

# **********************************************************
# move files to directories according to their CRS
for tif in tifs:
    fp = os.path.join(root, tif)
    
    if rioxr.open_rasterio(fp).rio.crs == crs11:
        newfp = os.path.join(fp_11, tif)
    if rioxr.open_rasterio(fp).rio.crs == crs10:
        newfp = os.path.join(fp_10, tif)
        
    os.rename(fp, newfp)
    
# **********************************************************
# SPLIT crs26910 into south and west coast

# list of files in crs10 folder
tifs = os.listdir(fp_10)

# directories for scenes in west and south coasts
fp_w = os.path.join(fp_10, 'west_coast')
fp_s = os.path.join(fp_10, 'south_coast')

# make directories
os.mkdir(fp_w)
os.mkdir(fp_s)

# open bounding box of SB county west coast and match crs
wcoast = gpd.read_file(os.path.join(os.getcwd(),'sb_west_coast','sb_west_coast.shp')).to_crs(crs10)

# move files to directories according to their coast location
for tif in tifs:
    
    fp = os.path.join(fp_10, tif)
    x = rioxr.open_rasterio(fp)
    bbox = box(*x.rio.bounds())
    
    if bbox.intersects(wcoast.geometry[0]):
        newfp = os.path.join(fp_w, tif)
    else:
        newfp = os.path.join(fp_s, tif)
        
    os.rename(fp, newfp)

In [19]:
# ----------------------------------------------------------
# merge crs26911 scenes
out_file = os.path.join(root,
                        'merged_crs26911_'+str(year)+'.tif')

in_files = os.path.join(root,
                        'crs_26911',
                        '*.tif')

command = make_command(out_file, in_files)
print(os.popen(command).read())

# ----------------------------------------------------------
# merge crs26910 South scenes
out_file = os.path.join(root,
                        'merged_crs26910_S_'+str(year)+'.tif')

in_files = os.path.join(root,
                        'crs_26910',
                        'south_coast',
                        '*.tif')

command = make_command(out_file, in_files)
print(os.popen(command).read())

# ----------------------------------------------------------
# merge crs26910 West scenes
out_file = os.path.join(root,
                        'merged_crs26910_W_'+str(year)+'.tif')

in_files = os.path.join(root,
                        'crs_26910',
                        'west_coast',
                        '*.tif')

command = make_command(out_file, in_files)
print(os.popen(command).read())

In [5]:
s_fp = os.path.join(root, 'merged_crs26910_S_' + str(year) + '.tif')
w_fp = os.path.join(root, 'merged_crs26910_W_' + str(year) + '.tif') 
r_fp = os.path.join(root, 'merged_crs26911_' + str(year) + '.tif')

In [6]:
# open crs26910 south raster and bounding box
s_raster = rioxr.open_rasterio(s_fp).squeeze()
s_bbox = box(*s_raster.rio.bounds())

# ------------- CLIP crs26910 W raster ----------
w_raster = rioxr.open_rasterio(w_fp).squeeze()
w_bbox = box(*w_raster.rio.bounds())

if (s_bbox.intersects(w_bbox)):
    new_bbox = w_bbox.difference(s_bbox)
    clipped_raster = w_raster.rio.clip([new_bbox]).squeeze()

    # save clipped raster
    with rasterio.open(
                os.path.join(root, 'merged_crs26910_W_' + str(year) + '_clip.tif'),  # file path
                'w',           # w = write
                driver = 'GTiff', # format
                height = clipped_raster.shape[0], 
                width = clipped_raster.shape[1],
                count = 1,  # number of raster bands in the dataset
                dtype = rasterio.uint8,
                crs = clipped_raster.rio.crs,
                transform = clipped_raster.rio.transform(),
            ) as dst:
                dst.write(clipped_raster.astype(rasterio.uint8), 1)
    os.remove(w_fp)

In [ ]:
# ------------- CLIP crs26911 raster ----------
# open crs26911 raster and bouding box
r_raster = rioxr.open_rasterio(r_fp).squeeze()
r_bbox = box(*r_raster.rio.bounds())

# match crs of bounding boxes
s_gdf = gpd.GeoDataFrame({'geometry': [s_bbox]}, crs= s_raster.rio.crs )
s_gdf = s_gdf.to_crs(r_raster.rio.crs)
s_bbox = s_gdf.geometry[0]

if (s_bbox.intersects(r_bbox)):
    # take difference of r_bbox and s_bbox
    new_bbox = r_bbox.difference(s_bbox)

    # clip crs26911 raster
    clipped_raster = r_raster.rio.clip([new_bbox]).squeeze()

    # save clipped raster
    with rasterio.open(
                os.path.join(root, 'merged_crs26911_' + str(year) + '_clip.tif'),  # file path
                'w',           # w = write
                driver = 'GTiff', # format
                height = clipped_raster.shape[0], 
                width = clipped_raster.shape[1],
                count = 1,  # number of raster bands in the dataset
                dtype = rasterio.uint8,
                crs = clipped_raster.rio.crs,
                transform = clipped_raster.rio.transform(),
            ) as dst:
                dst.write(clipped_raster.astype(rasterio.uint8), 1)
    os.remove(r_fp)